In [17]:
from PyPDF2 import PdfReader
import os
import sys
import openai

In [8]:
dir = '/Users/katiegreed/Documents/Duke AI Masters/AIPI 590 - Large Language Models/HW/Assignment2/Docs/'

data = []
data_names = []
for doc in os.listdir(dir):
    print(doc)
    try:
        reader = PdfReader(dir + doc)
        content = ''
        for page in reader.pages:
            content += page.extract_text()
    except:
        print(f"Could not read {doc}. Skipping...")
    data.append(content)
    data_names.append(doc)

LEADERSHIP OF THE COXSWAIN .pdf
.DS_Store
Could not read .DS_Store. Skipping...
Rules_Of_Rowing_2023_FINAL_PDF.pdf
Coaching-the-Coxswain-EXTRACT.pdf
Could not read Coaching-the-Coxswain-EXTRACT.pdf. Skipping...
TCC Coxswain Handbook 4.pdf
2022-23D1WCR_PreChampionshipManual.pdf
FiveKeystoCompetitiveCoxing.pdf
Coxswain Commands.pdf


In [9]:
data[0]

' \nContact Information:  Yaz Farooq , Head Coach Women\'s Rowing  \nStanford University,  Tel: (650) 387-8969 (cell)  \nyfarooq@stanford.edu      www.gostanford.com       \nwww.COXSWAINation.com  LEADERSHIP OF THE COXSWAIN \n \nYasmin Farooq \n \nGoal:  To develop leaders who will implement your coaching philosophies and strategies \nin practice and on race day.  Discussion will explore the difference between coxswain and rower leaders, making the most of both, a nd how they can work together to create a \ndynamic and productive environment for the team. \nOutline: \n \n1. Determining the actual ‘roles’ of coxswain and rower leaders \n \n2. Walking the Tightrope: Day to Day Leadership \nThe cox may direct the crew in practices and races, but s/he is not performing the \nphysical tasks.  How the cox can support the coach and team and still maintain the \nposition of “teammate.” \n \n3. Determining how much responsibility to  assign the coxswain in practice \n \n4. The importance of “Qu

In [10]:
data_names

['LEADERSHIP OF THE COXSWAIN .pdf',
 '.DS_Store',
 'Rules_Of_Rowing_2023_FINAL_PDF.pdf',
 'Coaching-the-Coxswain-EXTRACT.pdf',
 'TCC Coxswain Handbook 4.pdf',
 '2022-23D1WCR_PreChampionshipManual.pdf',
 'FiveKeystoCompetitiveCoxing.pdf',
 'Coxswain Commands.pdf']

In [11]:
'''Create chunks of text that can be processed and vectorized.
Want the chunks to end at the end of a senetence and the end of a paragraph.

Parameters: 
text - a string to be chunked 
Returns: a list of chunks of text
'''
def create_chunks(text, chunk_size=500):
    # Initialize variables
    chunks = []
    current_chunk = ''
    current_size = 0
    
    # Split the text into paragraphs
    paragraphs = text.split('\n\n')

    # Iterate through each paragraph
    for paragraph in paragraphs:
        # Split the paragraph into sentences
        sentences = paragraph.split('. ')
        
        # Iterate through each sentence
        for sentence in sentences:
            # Check if adding the sentence to the current chunk would exceed the chunk size
            if len(current_chunk) + len(sentence) + 2 <= chunk_size:  # Add 2 for the period and space
                # Add the sentence to the current chunk
                if current_chunk:
                    current_chunk += '. ' + sentence
                else:
                    current_chunk += sentence
                current_size += len(sentence) + 2  # Update the current size
            else:
                # Add the current chunk to the list of chunks
                chunks.append(current_chunk)
                # Reset the current chunk and size to start a new chunk
                current_chunk = sentence
                current_size = len(sentence)
        
        # Add the last chunk of the paragraph
        if current_chunk:
            chunks.append(current_chunk)
            current_chunk = ''
            current_size = 0
    
    # Return the list of chunks
    print(f'Created {len(chunks)} chunks')
    return chunks

chunks = create_chunks(data[0])
for i in range(5):
    print(f'Chunk {i+1} of {len(chunks)}')
    print(chunks[i])
    print('--------\n')

Created 5 chunks
Chunk 1 of 5
 
Contact Information:  Yaz Farooq , Head Coach Women's Rowing  
Stanford University,  Tel: (650) 387-8969 (cell)  
yfarooq@stanford.edu      www.gostanford.com       
www.COXSWAINation.com  LEADERSHIP OF THE COXSWAIN 
 
Yasmin Farooq 
 
Goal:  To develop leaders who will implement your coaching philosophies and strategies 
in practice and on race day
--------

Chunk 2 of 5
 Discussion will explore the difference between coxswain and rower leaders, making the most of both, a nd how they can work together to create a 
dynamic and productive environment for the team. 
Outline: 
 
1. Determining the actual ‘roles’ of coxswain and rower leaders 
 
2. Walking the Tightrope: Day to Day Leadership 
The cox may direct the crew in practices and races, but s/he is not performing the 
physical tasks
--------

Chunk 3 of 5
 How the cox can support the coach and team and still maintain the 
position of “teammate.” 
 
3. Determining how much responsibility to  assign th

In [31]:
'''
Creates embeddings for the chuncks of text using OpenAI's text embedding model. 
Connects to the Azure database and stores the embeddings in a table.

Parameters:
chunks - a list of strings to be embedded
table_name - the name of the table to store the embeddings

Returns: None
'''
def create_embeddings(chunks, table_name, model="text-embedding-3-small"):
    # Connect to Azure database

    # Create embeddings for each chunk
    client = openai.Client()
    embeddings = []
    for chunk in chunks:
        result = client.embeddings.create(input=chunk, model=model)
        embeddings.append(result.data[0].embedding)
    
    print(f'Created {len(embeddings)} embeddings')
    print(embeddings[0])
    return embeddings

embeddings = create_embeddings(chunks, 'chunk_embeddings')

Created 5 embeddings
[0.008944387547671795, -0.003129916964098811, 0.04018170386552811, 0.031125973910093307, -0.007082520052790642, -0.02583109401166439, -0.005081476178020239, -0.016849592328071594, -0.008962944149971008, -0.04812402278184891, 0.02555892802774906, -0.0251011922955513, -0.010138209909200668, -0.026622852310538292, 0.03718786686658859, 0.01378153171390295, 0.007806235924363136, -0.018470222130417824, 0.02667233720421791, 0.049732279032468796, 0.012198016978800297, -0.01660216972231865, -0.02610326185822487, -0.009556762874126434, 0.02555892802774906, 0.0035845590755343437, 0.03820230811834335, 0.015587729401886463, 0.012977403588593006, -0.05517561361193657, 0.014338236302137375, -0.03528270125389099, -0.04339821636676788, -0.022070245817303658, 0.024643458425998688, 0.007997989654541016, -0.0018077435670420527, 0.005248487461358309, -0.005659830290824175, 0.013855759054422379, 0.0061268433928489685, -0.028998125344514847, 0.005721686407923698, 0.04037964344024658, 0.0

In [25]:
def embed_query(query, model="text-embedding-3-small"):
    # Connect to Azure database

    # Create an embedding for the query
    client = openai.Client()
    result = client.embeddings.create(input=query, model=model)
    return result.data[0].embedding

In [37]:
'''
Semantic search the embeddings using cosine similarity to locate the most similar chunk given a query.
Parameters:

'''
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def max_cosine_similarity(query_embedding, doc_embeddings):
    # Convert query embedding to numpy array
    query_embedding = np.array(query_embedding).reshape(1, -1)
    
    max_similarity = float("-inf")
    for index, doc_embedding in enumerate(doc_embeddings):
        # Convert document embedding to numpy array and reshape
        doc_embedding = np.array(doc_embedding).reshape(1, -1)
        
        # Calculate cosine similarity between query and document
        similarity_score = cosine_similarity(query_embedding, doc_embedding)[0][0]
        
        # Update max similarity if necessary
        if similarity_score > max_similarity:
            max_similarity = similarity_score
            best_doc_embedding = index
    
    return max_similarity, best_doc_embedding

max_sim, best_doc = max_cosine_similarity(embed_query('How to create a race strategy as a coxswain?'), embeddings)
print(f'Max similarity: {max_sim}')
print(f'Best document: {best_doc}')
print(chunks[best_doc])

Max similarity: 0.7741745810909757
Best document: 3
The coxwain and implementation of strategy in a race scenario 
Determining how much flexibility to give your coxswain with strategy and tactical calls. 
 
7. Balance and Professionalism 
Managing the flow of information among coach, coxswain and rowers. 
 
 "As a coxswain, I concentrated most on knowing the people in my boat - why they were rowing, why they came down to the boathouse, what made them tick
